In [ ]:
import numpy as np
import pandas as pd
import urllib.request
import re
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, TFBertModel
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Embedding, Bidirectional
from sklearn.metrics import classification_report


In [ ]:
!pip


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install ten

In [ ]:
# BERT 토크나이저 및 모델 불러오기
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
bert_model = TFBertModel.from_pretrained('bert-base-multilingual-cased')

# 데이터 불러오기 및 중복 제거
try:
    urllib.request.urlretrieve("https://raw.githubusercontent.com/tonykorea99/Spam-alart/main/OriginData2.csv", filename='OriginData2.csv')
    data = pd.read_csv('OriginData2.csv', encoding='utf-8')
    data = data.sample(frac=1, random_state=42)
    print('총 샘플 수:', len(data))
except Exception as e:
    print('에러 발생:', e)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

총 샘플 수: 87002


In [ ]:

data = data[['v1', 'v2']]
data['v1'] = data['v1'].replace(['ham', 'spam'], [0, 1])
data.drop_duplicates(subset=['v2'], inplace=True)
print('중복 제거 후의 전체 샘플 수:', len(data))

# 데이터 전처리: 한글만 남기고 나머지는 제거
def remove_non_korean(text):
    return re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣]', ' ', text)

data['v2'] = data['v2'].apply(remove_non_korean)


중복 제거 후의 전체 샘플 수: 84726


In [ ]:


# BERT 입력 형식으로 변환
max_length = 100  # 최대 시퀀스 길이
input_ids_list = []
attention_masks_list = []

for sent in data['v2']:
    encoded_dict = tokenizer.encode_plus(
                        sent,
                        add_special_tokens=True,
                        max_length=max_length,
                        padding='max_length',
                        truncation=True,
                        return_attention_mask=True,
                        return_tensors='tf',
                   )

    input_ids_list.append(encoded_dict['input_ids'])
    attention_masks_list.append(encoded_dict['attention_mask'])


In [ ]:

input_ids = tf.concat(input_ids_list, axis=0)

# 레이블 데이터
labels = data['v1'].values

# 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(
    input_ids.numpy(), labels, test_size=0.2, random_state=42)

# 학습 데이터에서 다시 검증 데이터 분리
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=25)


In [ ]:
# 전체 데이터셋에서 정상과 비정상 데이터 개수 확인
num_normal = data['v1'].value_counts()[0]
num_abnormal = data['v1'].value_counts()[1]

print(f'전체 데이터셋 - 정상 메시지 개수: {num_normal}, 비정상 메시지 개수: {num_abnormal}')

# 학습 데이터에서 정상과 비정상 데이터 개수 확인
num_normal_train = np.sum(y_train == 0)
num_abnormal_train = np.sum(y_train == 1)

print(f'학습 데이터 - 정상 메시지 개수: {num_normal_train}, 비정상 메시지 개수: {num_abnormal_train}')

# 검증 데이터에서 정상과 비정상 데이터 개수 확인
num_normal_val = np.sum(y_val == 0)
num_abnormal_val = np.sum(y_val == 1)

print(f'검증 데이터 - 정상 메시지 개수: {num_normal_val}, 비정상 메시지 개수: {num_abnormal_val}')

# 테스트 데이터에서 정상과 비정상 데이터 개수 확인
num_normal_test = np.sum(y_test == 0)
num_abnormal_test = np.sum(y_test == 1)

print(f'테스트 데이터 - 정상 메시지 개수: {num_normal_test}, 비정상 메시지 개수: {num_abnormal_test}')

전체 데이터셋 - 정상 메시지 개수: 72410, 비정상 메시지 개수: 12316
학습 데이터 - 정상 메시지 개수: 46276, 비정상 메시지 개수: 7948
검증 데이터 - 정상 메시지 개수: 11576, 비정상 메시지 개수: 1980
테스트 데이터 - 정상 메시지 개수: 14558, 비정상 메시지 개수: 2388


In [ ]:



# GRU 모델 정의
model_gru = Sequential()
model_gru.add(Embedding(input_dim=len(tokenizer.vocab), output_dim=200, input_length=max_length))
model_gru.add(Bidirectional(GRU(128)))
model_gru.add(Dense(1, activation='sigmoid'))

# 모델 컴파일
model_gru.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 모델 훈련
history = model_gru.fit(
    x=X_train,
    y=y_train,
    epochs=5,
    batch_size=64,
    validation_data=(X_val, y_val)  # 검증 데이터 사용
)



Epoch 1/5
848/848 [==============================] - 352s 405ms/step - loss: 0.0264 - accuracy: 0.9907 - val_loss: 0.0071 - val_accuracy: 0.9982
Epoch 2/5
848/848 [==============================] - 333s 392ms/step - loss: 0.0020 - accuracy: 0.9995 - val_loss: 0.0034 - val_accuracy: 0.9992
Epoch 3/5
848/848 [==============================] - 324s 383ms/step - loss: 0.0012 - accuracy: 0.9996 - val_loss: 0.0023 - val_accuracy: 0.9995
Epoch 4/5
848/848 [==============================] - 318s 375ms/step - loss: 5.7921e-04 - accuracy: 0.9998 - val_loss: 0.0025 - val_accuracy: 0.9991
Epoch 5/5
848/848 [==============================] - 320s 377ms/step - loss: 2.7084e-04 - accuracy: 0.9999 - val_loss: 0.0024 - val_accuracy: 0.9996


In [ ]:

# 모델 평가
y_pred = model_gru.predict(X_test)
y_pred = (y_pred > 0.5).astype(int)

# 정확도 및 다양한 평가 지표 출력
loss, accuracy = model_gru.evaluate(X_test, y_test)
print(f'Loss: {loss}, Accuracy: {accuracy}')

# 다양한 평가 지표 출력
print(classification_report(y_test, y_pred))


In [ ]:
# 모델 저장
model_path = '/content/drive/MyDrive/smishing_model_grumodi.h5'
model_gru.save(model_path)


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
import matplotlib.pyplot as plt

def plot_learning_curve(history):
    # 훈련 및 검증 손실 데이터 추출
    train_loss = history.history['loss']
    val_loss = history.history['val_loss']

    # 훈련 및 검증 정확도 데이터 추출
    train_accuracy = history.history['accuracy']
    val_accuracy = history.history['val_accuracy']

    # 에포크 번호 추출
    epochs = range(1, len(train_loss) + 1)

    # 손실 그래프
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_loss, 'b-', label='Training Loss')
    plt.plot(epochs, val_loss, 'r-', label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    # 정확도 그래프
    plt.subplot(1, 2, 2)
    plt.plot(epochs, train_accuracy, 'b-', label='Training Accuracy')
    plt.plot(epochs, val_accuracy, 'r-', label='Validation Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.tight_layout()
    plt.show()

# 학습 곡선 시각화
plot_learning_curve(history)